In [71]:
import urllib.request
import requests
import re
from tika import parser
from tqdm import tqdm
import pandas as pd
import itertools
import numpy as np
from collections import Counter
import datetime
import os

In [72]:
def remov_duplicates(input):
    input = input.split(" ")
    c = Counter(input)
    s = "".join(c.keys())
    s_double = [key for key, value in c.items() if value==4 and key!='']*2
    if s_double:
        s="".join(s_double)
    return(s)

def has_numbers(input):
    digit_count = sum(char.isdigit() for char in input)
    if digit_count>1: 
        return True 
    else: 
        return False
    
def clean_data(i):
    i = re.sub("[5-9]\s+P M", "", i)
    i = re.sub('[A-Z]','', i)
    i = i.replace(""," ")
    i = remov_duplicates(i)
    return(i)

days = ['TUES', 'WED', 'THUR', 'FRI', 'SAT - MAT', 'SAT - EVE', 'SUN']
exclude = ['75TH ANNIVERSARY']

fall_months = ['SEPT', 'OCT']
winter_months = ['JAN', 'FEB']
spring_months = ['APR', 'MAY', "JUNE"]

months = {
        'fall': fall_months,
        'winter': winter_months,
        'spring': spring_months
    }

In [164]:
raw_files = os.listdir(os.getcwd()+"/raw_data")
for file in raw_files:
    print(file)
    pdf = f'./raw_data/{file}'
    file_type = re.search('Fall|Winter|Spring', file).group(0).lower()
    
    rawText = parser.from_file(pdf)
    rawList = rawText['content'].splitlines()
    rawList = [i for i in rawList if len(i)>0 and i not in exclude]

    ### join names that are split
    join_ind = []
    exceptions = ['Tarantella ', 'Opus 19/The Dreamer ', 'NEW COMBINATIONS ', 'ALL ROBBINS ']
    for ind, i in enumerate(rawList): 
        if i.endswith(' ') and i not in exceptions and not has_numbers(i):
            join_ind.append(ind)
            join_ind.append(ind+1)

    ### join names that are split
    l = sorted(set(join_ind))
    group = []
    val = 0
    for ind, i in enumerate(l):
        if ind==0:
            group.append(val)
        else:
            if l[ind] - l[ind-1] > 1:
                val += 1
            group.append(val)

    g_df = pd.DataFrame({'ind':l, 'group':group})
    ind_to_pop = []
    for ind in range(g_df.shape[0]):
        min_ind = g_df[g_df.group==ind].ind.min()
        max_ind = g_df[g_df.group==ind].ind.max()+1
        if not np.isnan(min_ind):
            new_string = ''.join(rawList[min_ind:max_ind])
            rawList[min_ind] = new_string
            ind_to_pop.append([i for i in range(min_ind+1, max_ind, 1)])

    ind_to_pop = list(itertools.chain(*ind_to_pop))
    rawList = [i for j, i in enumerate(rawList) if j not in ind_to_pop]
    
    ## search for index with year
    for ind_break, i in enumerate(rawList):
        if re.search('202[3-9]{1}', i):
            break

    rawList = rawList[ind_break+1:-1]
    dates = []
    dates_ind = []
    for ind, i in enumerate(rawList):
        if has_numbers(i):
            dates.append(clean_data(i))
            dates_ind.append(ind)
            
    program_groups = sorted([i for i in range(0, len(dates), 1)] * round(len(dates)/7))
    program_groups_ind = [i for i in range(0, 4, 1)] * len(program_groups)
    
    
    program_names = []
    pieces = []
    for ind in range(len(dates_ind)):
        min_ind = dates_ind[ind]+1
        try:
            max_ind = dates_ind[ind+1]
        except:
            max_ind = -1
        subset = rawList[min_ind:max_ind]
        program_name = "".join([i for i in subset if i.isupper()])
        s = ", ".join([re.sub(' +', ' ', i).strip() for i in subset if i not in program_name])
        program_names.append(program_name)
        pieces.append(s)
        
    d = pd.DataFrame({
        'date':dates,
        'day':list(np.repeat(days, 4)),
        'program name':program_names,
        'repertory':pieces
    })

    month_lis = []
    for day in days:
        l = d[d.day==day].date.tolist()
        month_ind = 0
        for i in range(len(l)):
            if int(l[i-1])-int(l[i])>15:
                month_ind += 1
            month_lis.append(months.get(file_type)[month_ind])

    d['month'] = month_lis
    d['month_date'] = [sub[:3] for sub in d['month']]+d['date']
    d['date'] = [datetime.datetime.strptime(i, '%b%d').strftime('%b %d') for i in d['month_date'].tolist()]
    d = d.drop(['month', 'month_date'],axis=1)
    
    d.to_csv(f'./output_data/{file}.csv', index=False)

NYCB_2023 Fall Season.pdf
14
['19', '26', '3', '10', '20', '27', '4', '11', '21', '28', '5', '12', '22', '29', '6', '13', '30', '23', '7', '14', '23', '30', '7', '14', '24', '1', '8', '15']
NYCB_2024 Spring Season.pdf
14
['23', '30', '7', '14', '24', '1', '8', '15', '25', '2', '9', '16', '26', '3', '10', '4', '27', '11', '18', '27', '4', '11', '18', '28', '5', '12', 'olol', '21', '22', '23', '24', '25', '26', '28', '31', '1', '1', '2', '19', '17', '25', '29', '30', 'bibi']


ValueError: All arrays must be of the same length

In [19]:
### LINKS TO YOUTUBE VIDEOS SHOWING/DISCUSSING THESE PIECES THIS SEASON?

## next steps -- need to search NYCB rep page for info on music, etc. more details on the piece